In [18]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
from random import randint
from collections import OrderedDict 
import numpy as np
import pandas as pd
import random
import sys
import io
import re
from unidecode import unidecode

In [19]:
songs = pd.read_csv('data/cardi-b-lyrics.csv')

In [20]:
def get_tokenized_lines(df):
    words = []
    
    for index, row in df['lyrics'].iteritems():
        row = str(row).lower()
        for line in row.split('|-|'):
            new_words = re.findall(r"\b[a-z']+\b", unidecode(line))
            words = words + new_words
        
    return words

In [21]:
all_lyric_lines = get_tokenized_lines(songs)

In [22]:
SEQ_LENGTH = 50 + 1
sequences = list()

for i in range(SEQ_LENGTH, len(all_lyric_lines)):
    seq = all_lyric_lines[i - SEQ_LENGTH: i]
    sequences.append(seq)

print('Total Sequences: %d' % len(sequences))

Total Sequences: 19353


In [23]:
def save_doc(lines, filename):
    for line in lines:
        data = ' '.join(line)
        
    '\n'.join(data)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [24]:
out_filename = 'sequences.txt'
save_doc(sequences, out_filename)

In [25]:
vocab = set(all_lyric_lines)
vocab = sorted(vocab)

word_to_index = {w: i for i, w in enumerate(vocab)}
index_to_word = {i: w for w, i in word_to_index.items()}
word_indices = [word_to_index[word] for word in vocab]
vocab_size = len(vocab)

print(word_to_index)
print('vocabulary size: {}'.format(vocab_size))

{"'s": 0, 'a': 1, 'aarp': 2, 'abort': 3, 'about': 4, 'above': 5, 'abraza': 6, 'abrupt': 7, 'account': 8, 'accounts': 9, 'ace': 10, 'acento': 11, 'across': 12, 'act': 13, 'actin': 14, 'acting': 15, 'activos': 16, 'actually': 17, 'addicted': 18, 'administer': 19, 'admiring': 20, 'adore': 21, 'afford': 22, "afisia'o": 23, 'afisiado': 24, 'after': 25, 'again': 26, 'ago': 27, 'ah': 28, 'aha': 29, 'ahora': 30, "ain't": 31, 'air': 32, 'airplane': 33, 'alante': 34, 'album': 35, 'albums': 36, 'algo': 37, 'alive': 38, 'all': 39, 'alla': 40, 'allow': 41, 'alone': 42, 'alright': 43, 'always': 44, 'am': 45, 'amar': 46, 'amends': 47, 'amigas': 48, 'ammunition': 49, 'amor': 50, 'an': 51, 'and': 52, 'andamo': 53, 'andamos': 54, 'ando': 55, 'andre': 56, 'andretti': 57, 'angel': 58, 'anger': 59, 'ano': 60, 'another': 61, 'answer': 62, 'any': 63, 'anybody': 64, 'anymore': 65, "apaga'o": 66, 'aparte': 67, 'apartments': 68, 'appetit': 69, 'aqui': 70, 'ar': 71, 'are': 72, "aren't": 73, 'armadillo': 74, 'arm

In [26]:
def get_tokenized_lines(lines, seq_len):
    tokenized = np.zeros((len(lines), seq_len))
    
    for r, line in enumerate(lines):
        for c, word in enumerate(line):
            tokenized[r, c] = word_to_index[word]

    return tokenized

In [27]:
tokenized_seq = get_tokenized_lines(sequences, SEQ_LENGTH)

In [28]:
tokenized_seq[:, -1].shape

(19353,)

In [29]:
from keras.utils import to_categorical

X, y = tokenized_seq[:, :-1], tokenized_seq[:, -1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = len(X[0])

In [30]:
print("X_shape", X.shape)
print("y_shape", y.shape)

X_shape (19353, 50)
y_shape (19353, 2341)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X, y, batch_size=128, epochs=200)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 50)            117050    
_________________________________________________________________
lstm_4 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 2341)              236441    
Total params: 504,391
Trainable params: 504,391
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
152/152 [==============================] - 19s 123ms/step - loss: 6.4580 - accuracy: 0.0435
Epoch

152/152 [==============================] - 19s 127ms/step - loss: 1.2100 - accuracy: 0.7116
Epoch 70/200
152/152 [==============================] - 22s 145ms/step - loss: 1.1828 - accuracy: 0.7170
Epoch 71/200
152/152 [==============================] - 18s 120ms/step - loss: 1.1711 - accuracy: 0.7206
Epoch 72/200
152/152 [==============================] - 17s 113ms/step - loss: 1.1463 - accuracy: 0.7225
Epoch 73/200
152/152 [==============================] - 23s 151ms/step - loss: 1.1244 - accuracy: 0.7261
Epoch 74/200
152/152 [==============================] - 23s 152ms/step - loss: 1.0996 - accuracy: 0.7352
Epoch 75/200
152/152 [==============================] - 28s 184ms/step - loss: 1.0743 - accuracy: 0.7429
Epoch 76/200
152/152 [==============================] - 19s 122ms/step - loss: 1.0603 - accuracy: 0.7443
Epoch 77/200
152/152 [==============================] - 16s 105ms/step - loss: 1.0320 - accuracy: 0.7484
Epoch 78/200
152/152 [==============================] - 16s 106ms/st

152/152 [==============================] - 19s 124ms/step - loss: 0.2236 - accuracy: 0.9508
Epoch 147/200
152/152 [==============================] - 20s 132ms/step - loss: 0.2211 - accuracy: 0.9495
Epoch 148/200
152/152 [==============================] - 20s 131ms/step - loss: 0.2464 - accuracy: 0.9415
Epoch 149/200
152/152 [==============================] - 23s 150ms/step - loss: 0.2448 - accuracy: 0.9440
Epoch 150/200
152/152 [==============================] - 18s 116ms/step - loss: 0.2040 - accuracy: 0.9561
Epoch 151/200
152/152 [==============================] - 19s 123ms/step - loss: 0.1863 - accuracy: 0.9606
Epoch 152/200
152/152 [==============================] - 18s 119ms/step - loss: 0.1741 - accuracy: 0.9636
Epoch 153/200
152/152 [==============================] - 24s 157ms/step - loss: 0.1608 - accuracy: 0.9685
Epoch 154/200
152/152 [==============================] - 23s 151ms/step - loss: 0.1754 - accuracy: 0.9638
Epoch 155/200
152/152 [==============================] - 23s

In [ ]:
model.save('word_model_cardi_b.h5')

In [56]:
from keras.models import load_model

model = load_model('word_model_eminem.h5')

In [57]:
def texts_to_sequences(texts, word_to_index):
    indices = np.zeros((1, len(texts)), dtype=int)
    
    for i, text in enumerate(texts):
        if text not in word_to_index:
            random = index_to_word[randint(0,vocab_size)]
            indices[:, i] = word_to_index[random]
        else:
            indices[:, i] = word_to_index[text]
        
    return indices

In [58]:
def my_pad_sequences(seq, maxlen):
    start = seq.shape[1] - maxlen
    
    return seq[:, start: start + maxlen]

In [59]:
def generate_seq(model, word_to_index, seq_length, n_words):
    generated = ''
    usr_input = input("Write the beginning of your poem, the Drake machine will complete it. Your input is: ")
    generated += usr_input 
    sys.stdout.write("\n\nHere is your poem: \n\n") 
    
    result = list()
    in_text = [None] * 51
    generated_list = generated.split()
    
    # if input is shorter than 51 words, fill the beginning with random words
    if(len(generated_list) < 51):
        end = len(generated_list)
        for i in range (51 - end):
            random = index_to_word[randint(0,vocab_size)]
            in_text[i] = random
            
        index = 0
        for i in range (51 - end, 51):
            in_text[i] = generated_list[index]
            index += 1

    # if input is longer than 51 words, only use the last 51 words
    if(len(generated_list) > 51):
        end = len(generated_list)
        in_text = generated_list[end-51:]

    # generate words based on last 50 words
    for _ in range(n_words):
        encoded = texts_to_sequences(in_text[1:], word_to_index)
        encoded = my_pad_sequences(encoded, maxlen=seq_length)
        
        yhat = model.predict_classes(encoded, verbose=0)
        out_word = ''
    
        for word, index in word_to_index.items():
            if index == yhat:
                out_word = word
                break
        
        in_text += ' ' + out_word
        result.append(out_word)
        
    return ' '.join(result)

In [60]:
generated = generate_seq(model, word_to_index, seq_length, 50)
print(generated)

Write the beginning of your poem, the Drake machine will complete it. Your input is: sdahfjashdfkjashd asdfh


Here is your poem: 

['coke', 'cookin', "problem's", 'headache', 'fifth', 'ovulating', 'birth', 'dear', 'neither', 'grips', 'aight', 'pacquiao', 'truthful', 'mann', 'babe', 'misconducts', 'extraterrestrial', 'dodgers', 'smack', 'ruin', 'jade', 'chasing', 'cars', 'poppins', 'tear', 'legendary', 'sorry', 'plenty', 'lewinsky', 'outdo', 'raps', 'growing', 'wide', 'sister', "you'ont", 'opposites', 'winks', 'glide', 'huge', 'southpaw', 'grasp', 'symbolic', 'harris', 'come', 'helped', 'muster', 'sit', 'beer', 'sweden', 'sdahfjashdfkjashd', 'asdfh']
envision fellatio fellow fellatio fellow denver clothes flippity christmas flippity cat triangle uh sa minimum fellatio fellow fellatio fellow fellatio groups resentment lazy hangs pectations jump pigeonholed ho's pinkett clothes pinkett clothes pinkett clothes pinkett lazy hangs bury idaho west pigeonholed fellatio fellow fellatio fellow 